In [19]:
%%pyspark

df = spark.read.load('abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_by_Title.csv', format='csv'
## If header exists uncomment line below
, header=True
)
display(df.limit(10))

StatementMeta(synasp1, 4, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 50f60c8d-c66e-41a5-8d2a-690157f3fb2e)

In [20]:
%%pyspark

# Show Schema
df.printSchema()

StatementMeta(synasp1, 4, 14, Finished, Available)

root
 |-- UsageClass: string (nullable = true)
 |-- CheckoutType: string (nullable = true)
 |-- MaterialType: string (nullable = true)
 |-- CheckoutYear: string (nullable = true)
 |-- CheckoutMonth: string (nullable = true)
 |-- Checkouts: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Creator: string (nullable = true)
 |-- Subjects: string (nullable = true)
 |-- Publisher: string (nullable = true)
 |-- PublicationYear: string (nullable = true)

In [21]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
capture_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
capture_container_name = 'capture' # fill in your container name
capture_relative_path = 'SeattlePublicLibrary/Checkouts_by_Title.csv' # fill in your relative folder path

capture_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (capture_container_name, capture_account_name, capture_relative_path)
print('Primary storage account path: ' + capture_adls_path)

StatementMeta(synasp1, 4, 15, Finished, Available)

Primary storage account path: abfss://capture@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/Checkouts_by_Title.csv

In [22]:
%%pyspark

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType, TimestampType
csvSchema = StructType([
  StructField('usage_class', StringType(), True),
  StructField('checkout_type', StringType(), True),
  StructField('material_type', StringType(), True), 
  StructField('checkout_year', IntegerType(), True),
  StructField('checkout_month', IntegerType(), True),
  StructField('checkouts', IntegerType(), True),   
  StructField('title', StringType(), True),
  StructField('creator', StringType(), True),
  StructField('subjects', StringType(), True),
  StructField('publisher', StringType(), True),
  StructField('publication_year', StringType(), True)   
])

CheckByTPI_capture_df = spark.read.format('csv').option('header', 'True').schema(csvSchema).load(capture_adls_path)

display(CheckByTPI_capture_df.limit(10))



StatementMeta(synasp1, 4, 16, Finished, Available)

SynapseWidget(Synapse.DataFrame, 68fd3507-9899-4747-85a4-92b0500ae640)

In [23]:
%%pyspark

from pyspark.sql.functions import to_date, to_timestamp, col, date_format, current_timestamp
df_final = (CheckByTPI_capture_df.withColumn('loadDate', date_format(current_timestamp(), 'M/d/y H:m:s a'))
                                 .withColumn("load_date", to_timestamp(col("loadDate"),"M/d/y H:m:s a")).drop("loadDate")
)

StatementMeta(synasp1, 4, 17, Finished, Available)

In [24]:
%%pyspark

# Show Schema
df_final.printSchema()

display(df_final.limit(10))


StatementMeta(synasp1, 4, 18, Finished, Available)

root
 |-- usage_class: string (nullable = true)
 |-- checkout_type: string (nullable = true)
 |-- material_type: string (nullable = true)
 |-- checkout_year: integer (nullable = true)
 |-- checkout_month: integer (nullable = true)
 |-- checkouts: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- subjects: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publication_year: string (nullable = true)
 |-- load_date: timestamp (nullable = true)



SynapseWidget(Synapse.DataFrame, 007c0bd4-e630-4de0-b403-0f264277753f)

In [25]:
%%pyspark

from pyspark.sql import SparkSession
from pyspark.sql.types import *

# Primary storage info
compose_account_name = 'spkaccelerjqvse6bhhchxi' # fill in your primary account name
compose_container_name = 'compose' # fill in your container name
compose_relative_path = 'SeattlePublicLibrary/CheckoutsByTitle/' # fill in your relative folder path

compose_adls_path = 'abfss://%s@%s.dfs.core.windows.net/%s' % (compose_container_name, compose_account_name, compose_relative_path)
print('Primary storage account path: ' + compose_adls_path)

StatementMeta(synasp1, 4, 19, Finished, Available)

Primary storage account path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitle/

In [26]:
%%pyspark

compose_parquet_path = compose_adls_path + 'checkouts.parquet'

print('parquet file path: ' + compose_parquet_path)

StatementMeta(synasp1, 4, 20, Finished, Available)

parquet file path: abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitle/checkouts.parquet

In [27]:
%%pyspark

df_final.write.parquet(compose_parquet_path, mode = 'overwrite')

StatementMeta(synasp1, 4, 21, Finished, Available)

In [28]:
%%sql

-- Create database SeattlePublicLibrary only if database with same name does not exist
CREATE DATABASE IF NOT EXISTS SeattlePublicLibrary

StatementMeta(synasp1, 4, 22, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>

In [29]:
%%sql

-- Create table CheckoutsByTitlePhysicalItemsschemafinal only if table with same name does not exist
CREATE TABLE IF NOT EXISTS SeattlePublicLibrary.checkouts_by_title
(usage_class STRING
,checkout_type STRING
,material_type STRING
,checkout_year INT
,checkout_month INT
,checkouts INT
,title STRING
,creator STRING
,subjects STRING
,publisher STRING
,publication_year STRING
,load_date TIMESTAMP
)
USING PARQUET OPTIONS (path 'abfss://compose@spkaccelerjqvse6bhhchxi.dfs.core.windows.net/SeattlePublicLibrary/CheckoutsByTitle/checkouts.parquet')

StatementMeta(synasp1, 4, 23, Finished, Available)

<Spark SQL result set with 0 rows and 0 fields>